In [2]:
import pandas as pd
import json
df=pd.read_json("./BuildingMathData.jsonl", lines=True)
df.head()

,instruction,output,category
0,Calculate the design occupant load for a 500 s...,"Reasoning: Per NYC Building Code Table 1004.5,...",Occupant Load
1,Calculate the design occupant load for a 1120 ...,"Reasoning: Per NYC Building Code Table 1004.5,...",Occupant Load
2,Calculate the design occupant load for a 1920 ...,"Reasoning: Per NYC Building Code Table 1004.5,...",Occupant Load
3,Calculate the design occupant load for a 3000 ...,"Reasoning: Per NYC Building Code Table 1004.5,...",Occupant Load
4,Calculate the design occupant load for a 570 s...,"Reasoning: Per NYC Building Code Table 1004.5,...",Occupant Load


In [10]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
import torch

model_id="google/gemma-3-1b-it"
# model_id="Qwen/Qwen3Guard-Gen-4B"
device="cpu"

tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(model_id, dtype="auto", device_map=device)

# Try restarting your kernel if having issues

In [69]:
SYSTEM_PROMPT = {
        "role": "system",
        "content": "You are a helpful AI assitant that helps engineers and real estate developers"
    }

USER_MESSAGES = [
    {
        "role": "user",
        "content": row.instruction
    }
    for _, row in df.iloc[:5].iterrows()
]

POST_MESSAGE = {
        "role": "assistant",
        "content": "Reasoning:      "
    }


PROMPTS = [
    [SYSTEM_PROMPT, USER_MSG, POST_MESSAGE]
    for USER_MSG in USER_MESSAGES
]

In [70]:
print(json.dumps(PROMPTS[0], indent=4))

[
    {
        "role": "system",
        "content": "You are a helpful AI assitant that helps engineers and real estate developers"
    },
    {
        "role": "user",
        "content": "Calculate the design occupant load for a 500 sq ft Library Reading Room in a new university building in Manhattan, per NYC 2022 Building Code."
    },
    {
        "role": "assistant",
        "content": "Reasoning:      "
    }
]


In [71]:
tokenized = tokenizer.apply_chat_template(
    PROMPTS,
    continue_final_message=True,
    padding=True,
    return_tensors="pt"
).to(device)

out = model.generate(tokenized, max_new_tokens=14)

In [72]:
decoded = tokenizer.batch_decode(out)
decoded

["<pad><bos><start_of_turn>user\nYou are a helpful AI assitant that helps engineers and real estate developers\n\nCalculate the design occupant load for a 500 sq ft Library Reading Room in a new university building in Manhattan, per NYC 2022 Building Code.<end_of_turn>\n<start_of_turn>model\nReasoning:\n\nOkay, let's calculate the design occupant load for a ",
 '<pad><pad><bos><start_of_turn>user\nYou are a helpful AI assitant that helps engineers and real estate developers\n\nCalculate the design occupant load for a 1120 sq ft Classroom in a new university building in Manhattan, per NYC 2022 Building Code.<end_of_turn>\n<start_of_turn>model\nReasoning:\n\nOkay, let’s calculate the design occupant load for a ',
 '<bos><start_of_turn>user\nYou are a helpful AI assitant that helps engineers and real estate developers\n\nCalculate the design occupant load for a 1920 sq ft Library Reading Room in a new university building in Manhattan, per NYC 2022 Building Code.<end_of_turn>\n<start_of_tu

In [81]:
labels = [(d.split("\nReasoning:")[-1]).strip() for d in decoded]
labels

["Okay, let's calculate the design occupant load for a",
 'Okay, let’s calculate the design occupant load for a',
 'Okay, let’s calculate the Design Occupant Load (DOL)',
 'Okay, let’s calculate the Design Occupant Load for a',
 "Okay, let's calculate the design occupant load for this library"]